In [1]:
import tensorflow as tf

In [3]:
# 加载并准备 MNIST 数据集。
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# 通过堆叠层来构建 tf.keras.Sequential 模型。
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# 模型返回一个包含 logits 或 log-odds 分数的向量。
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.23877537, -0.48739383, -0.51455384,  0.44338647, -0.12068474,
         0.39337003, -0.29573056,  0.7667253 ,  1.4478253 ,  0.6249935 ]],
      dtype=float32)

In [6]:
# tf.nn.softmax 函数将这些 logits 转换为每个类的概率
tf.nn.softmax(predictions).numpy()

array([[0.0823048 , 0.03981566, 0.03874881, 0.10099193, 0.05745312,
        0.09606492, 0.04822722, 0.13954392, 0.2757458 , 0.12110374]],
      dtype=float32)

In [8]:
#  losses.SparseCategoricalCrossentropy 为训练定义损失函数，它会接受 logits 向量和 True 索引，并为每个样本返回一个标量损失。
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.342731

In [10]:
# 开始训练之前，使用 Keras Model.compile 配置和编译模型。
# 将 optimizer 类设置为 adam，将 loss 设置为之前定义的 loss_fn 函数，并通过将 metrics 参数设置为 accuracy 来指定要为模型评估的指标。
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [11]:
# 训练并评估模型
# 使用 Model.fit 方法调整模型参数并最小化损失
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8590 - loss: 0.4753
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9546 - loss: 0.1487
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9664 - loss: 0.1120
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9722 - loss: 0.0898
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9763 - loss: 0.0725


In [12]:
# Model.evaluate 方法通常在 "Validation-set" 或 "Test-set" 上检查模型性能。准确度已经达到 98%。
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 1ms/step - accuracy: 0.9741 - loss: 0.0795


[0.07950030267238617, 0.9740999937057495]

In [13]:
# 让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.0034313e-06, 8.3621700e-08, 3.8041362e-06, 5.7293877e-05,
        1.1743572e-11, 1.6999047e-07, 8.5056169e-14, 9.9993324e-01,
        1.4261160e-06, 2.9155535e-06],
       [1.0371161e-06, 3.2048786e-04, 9.9962568e-01, 3.7004247e-05,
        5.9819494e-13, 1.5688403e-06, 1.6280610e-06, 7.9230147e-12,
        1.2550057e-05, 3.9792184e-12],
       [5.7072089e-07, 9.9826789e-01, 6.2222511e-04, 5.5728897e-06,
        1.4146705e-05, 1.7121588e-06, 1.8345245e-05, 8.4199850e-04,
        2.2660171e-04, 9.3637647e-07],
       [9.9997795e-01, 1.6689643e-09, 1.1883243e-06, 1.4957541e-08,
        2.4779695e-07, 1.0542544e-05, 3.3818167e-06, 1.7374266e-06,
        9.2458883e-08, 4.8101078e-06],
       [2.2634176e-05, 2.6112724e-08, 3.9244343e-05, 2.4833258e-07,
        9.9830484e-01, 2.9108969e-06, 8.4782851e-06, 4.0951640e-05,
        6.6040317e-05, 1.5146089e-03]], dtype=float32)>